# 🗺️ 14 — DIM_LOCAL (Gold)

- Lê `silver/2018_anonimizado.xlsx` e `silver/2019_anonimizado.xlsx`
- Constrói dimensão de local com UF e Município
- Preferência: UF/Município do Processo; fallback: Cadastro
- Exporta em `gold/output/dim_local.csv`


## 0) Imports

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

## 1) Paths robustos

In [ ]:
BASE_DIR = Path().resolve()   
OUT_DIR = BASE_DIR / "output"
OUT_DIR.mkdir(parents=True, exist_ok=True)

INPUT_FILES = [
    BASE_DIR / "2018_anonimizado.xlsx",
    BASE_DIR / "2019_anonimizado.xlsx",
]

print("📁 BASE_DIR:", BASE_DIR)
print("📥 INPUT_FILES:")
for f in INPUT_FILES:
    print(" -", f, "| existe?", f.exists())

print("📤 OUT_DIR:", OUT_DIR)


📁 BASE_DIR: C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold
📥 INPUT_FILES:
 - C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\2018_anonimizado.xlsx | existe? True
 - C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\2019_anonimizado.xlsx | existe? True
📤 OUT_DIR: C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\output


## 2) Ler Silver (2018 + 2019)

In [3]:
dfs = []
for f in INPUT_FILES:
    if not f.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {f}")
    tmp = pd.read_excel(f, dtype=str)
    tmp["fonte_arquivo"] = f.name
    dfs.append(tmp)

df = pd.concat(dfs, ignore_index=True)

print("✅ Linhas/Colunas consolidadas:", df.shape)
df.head()


✅ Linhas/Colunas consolidadas: (732261, 87)


,ULTIMO_PROCESSO,SITUACAO_DO_PROCESSO,IS_SEDE_EAD,NO_DO_PROCESSO,MODALIDADE,ANO_DO_PROTOCOLO,DATA,ORGAO,ATO,CATEGORIA_ATO,...,CINE_AREA_ESPECIFICA,CODIGO_AREA_GERAL_CINE,AREA_GERAL_CINE,CODIGO_AREA_DETALHADA_CINE,AREA_DETALHADA_CINE,CODIGO_AREA_ESPECIFICA_CINE,AREA_ESPECIFICA_CINE,ROTULO_CINE,AVALIACAO_OFICIAL,fonte_arquivo
0,NÃO,Aguardando Pagamento,N,200810426,EAD,2009,2009-02-26 00:00:00,NaN,Credenciamento EAD,Instituição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018_anonimizado.xlsx
1,NÃO,Aguardando Pagamento,N,200810426,EAD,2009,2009-02-26 00:00:00,NaN,Credenciamento EAD,Instituição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018_anonimizado.xlsx
2,NÃO,Aguardando Pagamento,N,200810426,EAD,2009,2009-02-26 00:00:00,NaN,Credenciamento EAD,Instituição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018_anonimizado.xlsx
3,NÃO,Aguardando Pagamento,S,200810426,EAD,2009,2009-02-26 00:00:00,NaN,Credenciamento EAD,Instituição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018_anonimizado.xlsx
4,NÃO,Arquivado,N,20070028,PRESENCIAL,2008,2008-09-26 00:00:00,SERES/DIREG/CGRERCES,Reconhecimento de Curso,Curso,...,Humanidades (exceto línguas),2,Artes e humanidades,223,Filosofia e ética,22,Humanidades (exceto línguas),Filosofia,Regulação,2018_anonimizado.xlsx


## 3) Funções auxiliares

In [ ]:

def norm_missing(s: pd.Series) -> pd.Series:
    x = s.astype(str).str.strip()
    return x.replace({"": np.nan, "nan": np.nan, "NAN": np.nan, "None": np.nan, "NONE": np.nan})

def dedup_most_complete(df_in: pd.DataFrame, key: str) -> pd.DataFrame:
    score = df_in.notna().sum(axis=1)
    return (df_in.assign(_score=score)
              .sort_values([key, "_score"], ascending=[True, False])
              .drop_duplicates(subset=[key], keep="first")
              .drop(columns=["_score"]))

def pick_first_existing(candidates, df_):
    return next((c for c in candidates if c in df_.columns), None)


## 4) Construir DIM_LOCAL

In [5]:
uf_proc = norm_missing(df["UF_PROCESSO"]) if "UF_PROCESSO" in df.columns else pd.Series([pd.NA]*len(df))
uf_cad  = norm_missing(df["UF_CADASTRO"]) if "UF_CADASTRO" in df.columns else pd.Series([pd.NA]*len(df))
mun_proc = norm_missing(df["MUNICIPIO_PROCESSO"]) if "MUNICIPIO_PROCESSO" in df.columns else pd.Series([pd.NA]*len(df))
mun_cad  = norm_missing(df["MUNICIPIO_CADASTRO"]) if "MUNICIPIO_CADASTRO" in df.columns else pd.Series([pd.NA]*len(df))

local = pd.DataFrame({
    "uf": uf_proc.fillna(uf_cad).str.upper(),
    "municipio": mun_proc.fillna(mun_cad).str.upper()
})

local["uf"] = local["uf"].replace({"": np.nan, "nan": np.nan, "NAN": np.nan, "None": np.nan, "NONE": np.nan})
local["municipio"] = local["municipio"].replace({"": np.nan, "nan": np.nan, "NAN": np.nan, "None": np.nan, "NONE": np.nan})

local["municipio"] = local["municipio"].fillna("NÃO INFORMADO")
local = local.dropna(subset=["uf"])

dim_local = local.drop_duplicates().sort_values(["uf","municipio"]).reset_index(drop=True)
dim_local["id_local"] = dim_local.index + 1
dim_local = dim_local[["id_local","uf","municipio"]]

print("✅ DIM_LOCAL pronta:", dim_local.shape)
dim_local.head(15)


✅ DIM_LOCAL pronta: (4266, 3)


,id_local,uf,municipio
0,1,AC,ACRELÂNDIA
1,2,AC,ASSIS BRASIL
2,3,AC,BRASILÉIA
3,4,AC,BUJARI
4,5,AC,CAPIXABA
5,6,AC,CRUZEIRO DO SUL
6,7,AC,EPITACIOLÂNDIA
7,8,AC,FEIJÓ
8,9,AC,JORDÃO
9,10,AC,MANOEL URBANO


## 5) Exportar

In [6]:
out_file = OUT_DIR / "dim_local.csv"
dim_local.to_csv(out_file, index=False, encoding="utf-8")
print("✅ Salvo em:", out_file)


✅ Salvo em: C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\output\dim_local.csv
